In [37]:
import pandas as pd
import numpy as np

## Worklist prioritization: Emergency Setting

In [38]:
## First, read in the file of the current worklist with the probabilities that your two algorithms have
## generated for the two types of findings you're most concerned with:

worklist = pd.read_csv('probabilities.csv')

In [39]:
worklist.head()

,Image_Type,Brain_bleed_probability,Aortic_dissection_probability
0,chest_xray,0.0,0.05
1,chest_xray,0.0,0.17
2,chest_xray,0.0,0.00
3,chest_xray,0.0,0.04
4,wrist_xray,0.0,0.00


### Following are the tasks that I need to perform:

1. Calculate the amount of time it will take before each image is read by the radiologist, given the patient arrival queue, assuming each image takes 6 minutes to read
2. Implement a heuristic that uses the probabilities returned by your two algorithms to re-order the priority read list, assuming that brain bleeds and aortic dissections are equally urgent
3. Calculate the time delta for each image between the initial and the re-ordered priority reads

### Task 1

In [40]:
time_to_read = np.arange(6,6*(len(worklist)+1),6)
print(time_to_read)

[  6  12  18  24  30  36  42  48  54  60  66  72  78  84  90  96 102 108
 114 120 126 132 138 144 150 156 162 168 174 180 186 192 198 204 210 216
 222 228 234 240 246 252 258 264 270 276 282 288 294 300 306 312 318 324
 330 336 342 348 354 360 366 372 378 384 390 396 402 408 414 420 426 432
 438 444 450 456 462 468 474 480 486 492 498 504 510 516 522 528 534 540
 546 552 558 564 570 576 582 588 594]


In [41]:
worklist['time_to_read'] = time_to_read

In [42]:
worklist.head()

,Image_Type,Brain_bleed_probability,Aortic_dissection_probability,time_to_read
0,chest_xray,0.0,0.05,6
1,chest_xray,0.0,0.17,12
2,chest_xray,0.0,0.00,18
3,chest_xray,0.0,0.04,24
4,wrist_xray,0.0,0.00,30


### Task 2

#### Here I am check, based on the probabilities, which one of the either is very likely to happen 

In [43]:
worklist['most_likely_to_happen'] = worklist[['Brain_bleed_probability', 
                                                      'Aortic_dissection_probability']].max(axis=1)

In [44]:
worklist.head()

,Image_Type,Brain_bleed_probability,Aortic_dissection_probability,time_to_read,most_likely_to_happen
0,chest_xray,0.0,0.05,6,0.05
1,chest_xray,0.0,0.17,12,0.17
2,chest_xray,0.0,0.00,18,0.00
3,chest_xray,0.0,0.04,24,0.04
4,wrist_xray,0.0,0.00,30,0.00


In [45]:
# Sort the images in decresing order of likelihood

worklist_prioritised = worklist.sort_values(by=['most_likely_to_happen'], ascending=False) 

In [46]:
worklist_prioritised.head()

,Image_Type,Brain_bleed_probability,Aortic_dissection_probability,time_to_read,most_likely_to_happen
25,head_ct,0.99,0.00,156,0.99
15,chest_xray,0.00,0.95,96,0.95
10,chest_xray,0.00,0.94,66,0.94
75,chest_xray,0.00,0.93,456,0.93
47,chest_xray,0.00,0.93,288,0.93


### Task 3

In [47]:
worklist_prioritised['time_to_read_after_reorder'] = np.arange(6,6*(len(worklist)+1),6)

In [48]:
worklist_prioritised.head()

,Image_Type,Brain_bleed_probability,Aortic_dissection_probability,time_to_read,most_likely_to_happen,time_to_read_after_reorder
25,head_ct,0.99,0.00,156,0.99,6
15,chest_xray,0.00,0.95,96,0.95,12
10,chest_xray,0.00,0.94,66,0.94,18
75,chest_xray,0.00,0.93,456,0.93,24
47,chest_xray,0.00,0.93,288,0.93,30


In [49]:
worklist_prioritised['time_delta'] = worklist_prioritised['time_to_read'] - worklist_prioritised['time_to_read_after_reorder']

In [50]:
worklist_prioritised.head()

,Image_Type,Brain_bleed_probability,Aortic_dissection_probability,time_to_read,most_likely_to_happen,time_to_read_after_reorder,time_delta
25,head_ct,0.99,0.00,156,0.99,6,150
15,chest_xray,0.00,0.95,96,0.95,12,84
10,chest_xray,0.00,0.94,66,0.94,18,48
75,chest_xray,0.00,0.93,456,0.93,24,432
47,chest_xray,0.00,0.93,288,0.93,30,258


### Answer the following questions based on your reprioritization:

1. If your algorithm's goal was to have brain bleeds read 30 minutes faster, how did it do?
2. If your algorithm's goal was to have aortic dissections read 15 minutes faster, how did it do?
3. Were there any cases where your algorithm made it worse for patients who needed an ASAP read? Could anything have been done about this?

### Answer 1

In [51]:
worklist_prioritised[(worklist_prioritised.time_delta) > 30 & (worklist_prioritised.Image_Type == 'head_ct')]

,Image_Type,Brain_bleed_probability,Aortic_dissection_probability,time_to_read,most_likely_to_happen,time_to_read_after_reorder,time_delta
25,head_ct,0.99,0.00,156,0.99,6,150
15,chest_xray,0.00,0.95,96,0.95,12,84
10,chest_xray,0.00,0.94,66,0.94,18,48
75,chest_xray,0.00,0.93,456,0.93,24,432
47,chest_xray,0.00,0.93,288,0.93,30,258
84,head_ct,0.91,0.00,510,0.91,36,474
95,head_ct,0.90,0.00,576,0.90,42,534
42,head_ct,0.89,0.00,258,0.89,48,210
59,head_ct,0.89,0.00,360,0.89,54,306
48,chest_xray,0.00,0.84,294,0.84,60,234


In [52]:
len(worklist_prioritised[(worklist_prioritised.time_delta > 30) & (worklist_prioritised.Image_Type == 'head_ct')])

14

We can clearly see that 14 head CTs were read 30 min faster.

### Answer 2

In [53]:
worklist_prioritised[(worklist_prioritised.time_delta > 15) & (worklist_prioritised.Image_Type == 'chest_xray')]

,Image_Type,Brain_bleed_probability,Aortic_dissection_probability,time_to_read,most_likely_to_happen,time_to_read_after_reorder,time_delta
15,chest_xray,0.0,0.95,96,0.95,12,84
10,chest_xray,0.0,0.94,66,0.94,18,48
75,chest_xray,0.0,0.93,456,0.93,24,432
47,chest_xray,0.0,0.93,288,0.93,30,258
48,chest_xray,0.0,0.84,294,0.84,60,234
38,chest_xray,0.0,0.83,234,0.83,66,168
62,chest_xray,0.0,0.82,378,0.82,72,306
87,chest_xray,0.0,0.82,528,0.82,78,450
44,chest_xray,0.0,0.81,270,0.81,84,186
85,chest_xray,0.0,0.79,516,0.79,90,426


In [54]:
len(worklist_prioritised[(worklist_prioritised.time_delta > 15) & (worklist_prioritised.Image_Type == 'chest_xray')])

28

We can clearly see that 28 chest xrays were read 15 min faster.